<a href="https://colab.research.google.com/github/shreyus3003/GenAI/blob/main/GLFNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from scipy.special import gamma #Import the gamma function

In [ ]:

def grunwald_letnikov_derivative(image, alpha=0.95, h=1):
    """
    Calculate the Grünwald–Letnikov fractional derivative for a 2D image.

    :param image: 2D numpy array representing the image
    :param alpha: Fractional order (default 0.95)
    :param h: Step size (default 1)
    :return: Fractional derivative of the image
    """
    # Get image dimensions
    rows, cols = image.shape

    # Initialize the derivative image
    derivative = np.zeros_like(image)

    # Compute Grünwald–Letnikov fractional derivative for each pixel
    for i in range(rows):
        for j in range(cols):
            sum_left = 0
            sum_right = 0
            for k in range(1, min(i, j) + 1):
                # Use gamma function for fractional factorial
                coefficient = (-1)**k * (gamma(alpha + 1) / (gamma(k + 1) * gamma(alpha - k + 1)))
                if i - k >= 0:
                    sum_left += coefficient * image[i - k, j]
                if j - k >= 0:
                    sum_right += coefficient * image[i, j - k]
            derivative[i, j] = (sum_left + sum_right) / (h**alpha)

    return derivative

In [ ]:
## Define the CNN model (GLFNet)

class GLFNet(nn.Module):
  def __init__(self):
    super(GLFNet, self).__init__()
    self.conv1 = nn.Conv2d(1,32, kernel_size=3, padding=1)
    self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
    self.relu = nn.ReLU()
    self.sigmoid = nn.Sigmoid()

  def forward(self,x):
    x = self.relu(self.conv1(x))
    x = self.sigmoid(self.conv2(x))
    return x

In [ ]:
## Data preparation: MNIST and CIFAR-10

# transformation for MNIST
transform_mnist = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: torch.tensor(grunwald_letnikov_derivative(x.numpy().squeeze(), alpha=0.95)).unsqueeze(0))
])

# transformation for CIFAR-10
transform_cifar = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: torch.tensor(grunwald_letnikov_derivative(x.numpy().squeeze(), alpha=0.95)).unsqueeze(0))
])

In [ ]:
# Download and load MNIST
train_dataset_mnist = datasets.MNIST(root = './data', train=True, download=True, transform=transform_mnist)
train_loader_mnist = torch.utils.data.DataLoader(train_dataset_mnist, batch_size=64, shuffle=True)
test_dataset_mnist = datasets.MNIST(root = './data', train=False, download=True, transform=transform_mnist)
test_loader_mnist = torch.utils.data.DataLoader(test_dataset_mnist, batch_size=1000, shuffle=False)

In [ ]:
# Download and load CIFAR-10

train_dataset_cifar = datasets.CIFAR10(root='./data', train=True, transform=transform_cifar, download=True)
train_loader_cifar = torch.utils.data.DataLoader(train_dataset_cifar, batch_size=64, shuffle=True)
test_data_cifar = datasets.CIFAR10(root='./data', train=False, transform=transform_cifar, download=True)
test_loader_cifar = torch.utils.data.DataLoader(test_data_cifar, batch_size=1000, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
import torch.nn.functional as F

In [ ]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print('GPU is available')
else:
  device = torch.device('cpu')

GPU is available


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GLFNet().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device).float()

        optimizer.zero_grad()
        output = model(data)

        # Ensure target is in the correct shape and type for BCELoss
        target = target.view(-1, 1).to(device).float()  # Ensure target is float and reshaped

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
for epoch in range(1,11):
  print(f'Training on MNIST dataset, Epoch {epoch}')
  train(model, device, train_loader_mnist, optimizer, criterion, epoch)

# for epoch in range(1,11):
#   print(f'Training on CIFAR-10 dataset, Epoch {epoch}')
#   train(model, device, train_loader_cifar, optimizer, criterion, epoch)


Training on MNIST dataset, Epoch 1


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
def test(model, device, test_loader):
  model.eval()
  test_loss = 0
  correct = 0
  with torch.no_grad():
    for data, target in test_loader:
      data, target = data.to(device), target.to(device).float()
      output = model(data)
      test_loss += criterion(output, target).item()
      pred = output.round()
      correct+= pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(test_loader.dataset)
  print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [ ]:
print("Testing on MNIST dataset")
test(model, device, test_loader_mnist)
# print("Testing on CIFAR-10 dataset